# Deploy HDFS

In [32]:
import pprint, time, sys

In [33]:
varClouderaPath = "/vagrant/notebooks/cloudera"
sys.path.append(varClouderaPath)

varClouderaPath

'/vagrant/notebooks/cloudera'

In [34]:
sys.path

['',
 '/opt/anaconda/lib/python27.zip',
 '/opt/anaconda/lib/python2.7',
 '/opt/anaconda/lib/python2.7/plat-linux2',
 '/opt/anaconda/lib/python2.7/lib-tk',
 '/opt/anaconda/lib/python2.7/lib-old',
 '/opt/anaconda/lib/python2.7/lib-dynload',
 '/opt/anaconda/lib/python2.7/site-packages',
 '/opt/anaconda/lib/python2.7/site-packages/Sphinx-1.4.1-py2.7.egg',
 '/opt/anaconda/lib/python2.7/site-packages/setuptools-23.0.0-py2.7.egg',
 '/opt/anaconda/lib/python2.7/site-packages/IPython/extensions',
 '/root/.ipython',
 '/vagrant/notebooks/cloudera',
 '/vagrant/notebooks/cloudera',
 '/vagrant/notebooks/cloudera',
 '/vagrant/notebooks/cloudera']

In [35]:
!free

             total       used       free     shared    buffers     cached
Mem:       3922528    2226748    1695780        164      58672    1689036
-/+ buffers/cache:     479040    3443488 
Swap:      1015804          0    1015804 


# Connect to Cloudera Manager

In [36]:
from cm_api.endpoints.services import ApiService
from cm_api.endpoints.services import ApiServiceSetupInfo
from cm_api.api_client import ApiResource

In [37]:
varFullHost = "pocd-cm581-dev-manager.poc-d.internal"

varAdminUser = "admin"
varAdminPassword = "admin"

varApiResource = ApiResource(varFullHost, version=10, username=varAdminUser, password=varAdminPassword)
varManager = varApiResource.get_cloudera_manager()
varManager

In [ ]:
# Get cluster

In [38]:
varClusterName = "POC-D Cluster"


In [39]:
varCluster = varApiResource.get_cluster(varClusterName)


In [40]:
varZK_Name = "ZooKeeper"

In [41]:
!mkdir -p /opt/cloudera/lib/hadoop-hdfs
!mkdir -p /tmp/.cloudera_health_monitoring_canary_files
!mkdir -p /opt/cloudera/log/hadoop-hdfs/audit

!chmod -R 777 /opt/cloudera/lib/hadoop-hdfs
!chmod -R 777 /tmp/.cloudera_health_monitoring_canary_files
!chmod -R 777 /opt/cloudera/log/hadoop-hdfs/audit

In [42]:
varHDFSServiceConfig = {
    'dfs_replication': 2,
    'dfs_permissions': False,
    'dfs_block_local_path_access_user': "impala,hbase,mapred,spark",
    'hdfs_hadoop_group_name': "hadoop",
    'process_groupname': "hdfs",
    'process_username': "hdfs",
    'hdfs_user_home_dir': "/opt/cloudera/lib/hadoop-hdfs",
    'audit_event_log_dir': "/opt/cloudera/log/hadoop-hdfs/audit",
    'navigator_client_max_num_audit_log': 10,
    'navigator_audit_log_max_file_size': 100,
    'firehose_hdfs_canary_directory': "/tmp/.cloudera_health_monitoring_canary_files",
    'zookeeper_service': varZK_Name,
    'enable_config_alerts': True,
}

varHDFSServiceConfig

{'audit_event_log_dir': '/opt/cloudera/log/hadoop-hdfs/audit',
 'dfs_block_local_path_access_user': 'impala,hbase,mapred,spark',
 'dfs_permissions': False,
 'dfs_replication': 2,
 'enable_config_alerts': True,
 'firehose_hdfs_canary_directory': '/tmp/.cloudera_health_monitoring_canary_files',
 'hdfs_hadoop_group_name': 'hadoop',
 'hdfs_user_home_dir': '/opt/cloudera/lib/hadoop-hdfs',
 'navigator_audit_log_max_file_size': 100,
 'navigator_client_max_num_audit_log': 10,
 'process_groupname': 'hdfs',
 'process_username': 'hdfs',
 'zookeeper_service': 'ZooKeeper'}

In [43]:
!mkdir -p /opt/cloudera/log/hadoop-hdfs
!chmod -R 777 /opt/cloudera/log/hadoop-hdfs


In [44]:
varHDFS_NameNode_Config = {
    'dfs_name_dir_list': "/dfs/nn",
    'oom_heap_dump_dir': "/tmp",
    'oom_heap_dump_enabled': False,
    'namenode_log_dir': "/opt/cloudera/log/hadoop-hdfs",
    'max_log_backup_index': 10,
    'max_log_size': 100,
    'log_threshold': "INFO",
    'namenode_java_heapsize': 268435456,
    'enable_config_alerts': True,
}

varHDFS_NameNode_Config

{'dfs_name_dir_list': '/dfs/nn',
 'enable_config_alerts': True,
 'log_threshold': 'INFO',
 'max_log_backup_index': 10,
 'max_log_size': 100,
 'namenode_java_heapsize': 268435456,
 'namenode_log_dir': '/opt/cloudera/log/hadoop-hdfs',
 'oom_heap_dump_dir': '/tmp',
 'oom_heap_dump_enabled': False}

In [45]:
!mkdir -p /opt/cloudera/log/hadoop-hdfs
!chmod -R 777 /opt/cloudera/log/hadoop-hdfs

In [46]:
varHDFS_SecondaryNameNode_Config = {
    'fs_checkpoint_dir_list': "/dfs/snn",
    'oom_heap_dump_dir': "/tmp",
    'oom_heap_dump_enabled': False,
    'secondarynamenode_log_dir': "/opt/cloudera/log/hadoop-hdfs",
    'max_log_backup_index': 10,
    'max_log_size': 100,
    'log_threshold': "INFO",
    'secondary_namenode_java_heapsize': 268435456,
    'enable_config_alerts': True,
}

varHDFS_SecondaryNameNode_Config

{'enable_config_alerts': True,
 'fs_checkpoint_dir_list': '/dfs/snn',
 'log_threshold': 'INFO',
 'max_log_backup_index': 10,
 'max_log_size': 100,
 'oom_heap_dump_dir': '/tmp',
 'oom_heap_dump_enabled': False,
 'secondary_namenode_java_heapsize': 268435456,
 'secondarynamenode_log_dir': '/opt/cloudera/log/hadoop-hdfs'}

In [47]:
!mkdir -p /opt/cloudera/log/hadoop-hdfs
!chmod -R 777 /opt/cloudera/log/hadoop-hdfs


In [48]:
varHDFS_DataNode_Config = {
    'dfs_data_dir_list': "/dfs/dn",
    'dfs_datanode_du_reserved': 10737418240,
    'dfs_datanode_failed_volumes_tolerated': 0,
    'dfs_datanode_data_dir_perm': 755,
    'oom_heap_dump_dir': "/tmp",
    'oom_heap_dump_enabled': False,
    'datanode_log_dir': "/opt/cloudera/log/hadoop-hdfs",
    'max_log_backup_index': 10,
    'max_log_size': 100,
    'log_threshold': "INFO",
    'datanode_java_heapsize': 268435456,
    'enable_config_alerts': True,
}

varHDFS_DataNode_Config

{'datanode_java_heapsize': 268435456,
 'datanode_log_dir': '/opt/cloudera/log/hadoop-hdfs',
 'dfs_data_dir_list': '/dfs/dn',
 'dfs_datanode_data_dir_perm': 755,
 'dfs_datanode_du_reserved': 10737418240,
 'dfs_datanode_failed_volumes_tolerated': 0,
 'enable_config_alerts': True,
 'log_threshold': 'INFO',
 'max_log_backup_index': 10,
 'max_log_size': 100,
 'oom_heap_dump_dir': '/tmp',
 'oom_heap_dump_enabled': False}

In [49]:
!mkdir -p /etc/hadoop
!chmod -R 777 /etc/hadoop

In [50]:
varHDFS_Gateway_Config = {
    'dfs_client_use_trash' : 'true',
    'client_config_root_dir' : "/etc/hadoop",
    'hdfs_client_java_heapsize' : 268435456,
    'enable_config_alerts': True,
}


varHDFS_Gateway_Config

{'client_config_root_dir': '/etc/hadoop',
 'dfs_client_use_trash': 'true',
 'enable_config_alerts': True,
 'hdfs_client_java_heapsize': 268435456}

In [51]:
varHDFS_Balancer_Config = {
    'balancer_java_heapsize': 268435456,
    'enable_config_alerts': True,
}

varHDFS_Balancer_Config

{'balancer_java_heapsize': 268435456, 'enable_config_alerts': True}

In [52]:
varManagementHost = "pocd-cm581-dev-man1.poc-d.internal"

varDataNodeHosts = [
    "pocd-cm581-dev-node1.poc-d.internal",
    "pocd-cm581-dev-node2.poc-d.internal",
    "pocd-cm581-dev-node3.poc-d.internal",
]

varDataNodeHosts

['pocd-cm581-dev-node1.poc-d.internal',
 'pocd-cm581-dev-node2.poc-d.internal',
 'pocd-cm581-dev-node3.poc-d.internal']

In [58]:

def deploy_hdfs(cluster, hdfs_service_name, hdfs_config, hdfs_nn_service_name, 
                hdfs_nn_host, hdfs_nn_config, 
                hdfs_snn_host, hdfs_snn_config, 
                hdfs_dn_hosts, hdfs_dn_config, 
                hdfs_bal_host, hdfs_bal_config, 
                hdfs_gw_hosts, hdfs_gw_config):
    
    hdfs_service = cluster.create_service(hdfs_service_name, "HDFS")
    hdfs_service.update_config(hdfs_config)
    
    nn_service_pattern = "{0}-" + hdfs_nn_service_name
    hdfs_service.create_role(nn_service_pattern.format(hdfs_service_name), "NAMENODE", hdfs_nn_host)
    hdfs_service.create_role("{0}-snn".format(hdfs_service_name), "SECONDARYNAMENODE", hdfs_snn_host)

    hdfs_service.create_role("{0}-bal".format(hdfs_service_name), "BALANCER", hdfs_bal_host)
    
    datanode = 0
    for host in hdfs_dn_hosts:
        datanode += 1
        hdfs_service.create_role("{0}-dn-".format(hdfs_service_name) + str(datanode), "DATANODE", host)
    
    gateway = 0
    for host in hdfs_gw_hosts:
        gateway += 1
        hdfs_service.create_role("{0}-gw-".format(hdfs_service_name) + str(gateway), "GATEWAY", host)

    hdfs_service.get_role_config_group("HDFS-NAMENODE-BASE").update_config(hdfs_nn_config)
    hdfs_service.get_role_config_group("HDFS-DATANODE-BASE").update_config(hdfs_dn_config)
    hdfs_service.get_role_config_group("HDFS-SECONDARYNAMENODE-BASE").update_config(hdfs_snn_config)
    hdfs_service.get_role_config_group("HDFS-GATEWAY-BASE").update_config(hdfs_gw_config)
    hdfs_service.get_role_config_group("HDFS-BALANCER-BASE").update_config(hdfs_bal_config)
    
    return hdfs_service


In [60]:
hdfs_service = deploy_hdfs(varCluster, "HDFS", varHDFSServiceConfig, "nn", 
    varManagementHost, varHDFS_NameNode_Config, 
    varManagementHost, varHDFS_SecondaryNameNode_Config, 
    varDataNodeHosts, varHDFS_DataNode_Config, 
    varManagementHost, varHDFS_Balancer_Config, 
    varDataNodeHosts, varHDFS_Gateway_Config)

hdfs_service

In [61]:
# Initializes HDFS - format the file system
def init_hdfs(hdfs_service, hdfs_name, timeout):
   cmd = hdfs_service.format_hdfs("{0}-nn".format(hdfs_name))[0]
   if not cmd.wait(timeout).success:
       print "WARNING: Failed to format HDFS, attempting to continue with the setup" 

In [63]:
# Format HDFS

In [62]:
init_hdfs(hdfs_service, "HDFS", 180)

In [ ]:
hdfs_service.create_hdfs_tmp()

In [ ]:
varTimeOutInSeconds = 180
cmd = varCluster.deploy_client_config()
if not cmd.wait(varTimeOutInSeconds).success:
    print "Failed to deploy client configs for {0}".format(varCluster.name)


In [64]:
varCluster.start().wait()